# Six Degrees of Kevin Bacon
**Introduction** - Six Degrees of Kevin Bacon is a game based on the "six degrees of separation"
concept, which posits that any two people on Earth are six or fewer acquaintance links apart. Movie
buffs challenge each other to find the shortest path between an arbitrary actor and prolific actor
Kevin Bacon. It rests on the assumption that anyone involved in the film industry can be linked
through their film roles to Bacon within six steps.
The analysis of social networks can be a computationally intensive task, especially when dealing with
large volumes of data. It is also a challenging problem to devise a correct methodology to infer an
informative social network structure. Here, we will analyze a social network of actors and actresses
that co-participated in movies. We will do some simple descriptive analysis, and in the end try to
relate an actor/actress’s position in the social network with the success of the movies in which they
participate.

#### Rules & Notes - Please take your time to read the following points:

1. The submission deadline shall be set for the 10th of June at 23:59.
2. It is acceptable that you **discuss** with your colleagues different approaches to solve each step of the problem set. You are responsible for writing your own code, and analysing the results. Clear cases of cheating will be penalized with 0 points in this assignment;
3. After review of your submission files, and before a mark is attributed, you might be called to orally defend your submission;
4. You will be scored first and foremost by the number of correct answers, secondly by the logic used in the trying to approach each step of the problem set;
5. Consider skipping questions that you are stuck in, and get back to them later;
6. Expect computations to take a few minutes to finish in some of the steps.
7. **IMPORTANT** It is expected you have developed skills beyond writting SQL queries. Any question where you directly write a SQL query (then for example create a temporary table and use spark.sql to pass the query) will receive a 25% penalty. Using the Spark syntax (for example dataframe.select("\*").where("conditions")) is acceptable and does not incur this penalty. Comment your code in a reasonable fashion.
8. **Questions** – Any questions about this assignment should be posted in the Forum@Moodle. The last class will be an open office session for anyone with questions concerning the assignment. 
9. **Delivery** - To fulfil this activity you will have to upload the following materials to Moodle:
    1. An exported IPython notebook. The notebook should be solved (have results displayed), but should contain all neccesary code so that when the notebook is run in databricks it should also replicate these results. This means the all data downloading and processing should be done in this notebook. It is also important you clearly indicate where your final answer to each question is when you are using multiple cells (for example you print "my final anwser is" before your answer or use cell comments). Please make sure to name your file in the following way: *[student_number1]_[student_number2]_submission.ipynb*. As an example: *19740001_197400010_submission.ipynb*
    2. **Delivery** - You will also need to provide a signed statement of authorship, which is present in the last page;
    3. It is recommended you read the whole assignment before starting.
    4. You can add as many cells as you like to answer the questions.
    5. You can make use of caching or persisting your RDDs or Dataframes, this may speed up performance.
    6. If you have trouble with graphframes in databricks (specifically the import statement) you need to make sure the graphframes package is installed on the cluster you are running. If you click home on the left, then click on the graphframes library, from where you can install the package on your cluster (check the graphframes checkbox and click install). Another installation option is using the JAR available on Moodle with the graphframes library.
10. **Note**: By including the name and student number of each group member  in the submission notebook, this will be considered as a declaration of authorship.

#### Data Sources and Description
We will use data from IMDB. You can download raw datafiles
from https://datasets.imdbws.com. Note that the files are tab delimited (.tsv) You can find a
description of the each datafile in https://www.imdb.com/interfaces/

## Questions
### Data loading and preperation
Review the file descriptions and load the necessary data onto your databricks cluser and into spark dataframes. You will need to use shell commands to download the data, unzip the data, load the data into spark. Note that the data might require parsing and preprocessing to be ready for the questions below.

**Hints** You can use 'gunzip' to unzip the .tz files. The data files will then be tab seperated (.tsv), which you can load into a dataframe using the tab seperated option instead of the comma seperated option we have typically used in class: `.option(“sep”,”\t”)`

> Load data

In [0]:
%sh

wget https://datasets.imdbws.com/name.basics.tsv.gz
wget https://datasets.imdbws.com/title.basics.tsv.gz
wget https://datasets.imdbws.com/title.ratings.tsv.gz
wget https://datasets.imdbws.com/title.crew.tsv.gz
wget https://datasets.imdbws.com/title.episode.tsv.gz
wget https://datasets.imdbws.com/title.principals.tsv.gz
wget https://datasets.imdbws.com/title.akas.tsv.gz


--2024-06-07 21:35:50--  https://datasets.imdbws.com/name.basics.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 99.84.66.78, 99.84.66.98, 99.84.66.41, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|99.84.66.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 267548102 (255M) [binary/octet-stream]
Saving to: ‘name.basics.tsv.gz’

     0K .......... .......... .......... .......... ..........  0% 8.87M 29s
    50K .......... .......... .......... .......... ..........  0% 9.40M 28s
   100K .......... .......... .......... .......... ..........  0% 10.6M 27s
   150K .......... .......... .......... .......... ..........  0% 36.7M 22s
   200K .......... .......... .......... .......... ..........  0% 16.7M 20s
   250K .......... .......... .......... .......... ..........  0% 49.0M 18s
   300K .......... .......... .......... .......... ..........  0% 26.0M 17s
   350K .......... .......... .......... .......... ..........  0% 20.0M 16s


> Unzip data

In [0]:
%sh

gunzip name.basics.tsv.gz
gunzip title.basics.tsv.gz
gunzip title.ratings.tsv.gz
gunzip title.crew.tsv.gz
gunzip title.episode.tsv.gz
gunzip title.principals.tsv.gz
gunzip title.akas.tsv.gz

gzip: title.basics.tsv already exists;	not overwritten


> Show the data

In [0]:
%%sh

ls; pwd

azure
conf
eventlogs
ganglia
hadoop_accessed_config.lst
logs
metastore_db
name.basics.tsv
preload_class.lst
title.akas.tsv
title.basics.tsv
title.basics.tsv.gz
title.crew.tsv
title.episode.tsv
title.principals.tsv
title.ratings.tsv
/databricks/driver


> Load the data in dataframes

In [0]:
df_title_akas = spark.read.option("sep", "\t").option("header", "true").option("inferSchema", "true").csv("file:/databricks/driver/title.akas.tsv")
df_title_basics = spark.read.option("sep", "\t").option("header", "true").option("inferSchema", "true").csv("file:/databricks/driver/title.basics.tsv")
df_title_crew = spark.read.option("sep", "\t").option("header", "true").option("inferSchema", "true").csv("file:/databricks/driver/title.crew.tsv")
df_title_episode = spark.read.option("sep", "\t").option("header", "true").option("inferSchema", "true").csv("file:/databricks/driver/title.episode.tsv")
df_title_principals = spark.read.option("sep", "\t").option("header", "true").option("inferSchema", "true").csv("file:/databricks/driver/title.principals.tsv")
df_title_ratings = spark.read.option("sep", "\t").option("header", "true").option("inferSchema", "true").csv("file:/databricks/driver/title.ratings.tsv")
df_name_basics = spark.read.option("sep", "\t").option("header", "true").option("inferSchema", "true").csv("file:/databricks/driver/name.basics.tsv")

> Verify the dataframe

In [0]:
df_title_akas.columns

Out[64]: ['titleId',
 'ordering',
 'title',
 'region',
 'language',
 'types',
 'attributes',
 'isOriginalTitle']

> Save dataframes to DBFS

In [0]:
df_name_basics.write.mode("overwrite").parquet("dbfs:/FileStore/tables/name_basics.parquet")
df_title_basics.write.mode("overwrite").parquet("dbfs:/FileStore/tables/title_basics.parquet")
df_title_akas.write.mode("overwrite").parquet("dbfs:/FileStore/tables/title_akas.parquet")
df_title_crew.write.mode("overwrite").parquet("dbfs:/FileStore/tables/title_crew.parquet")
df_title_principals.write.mode("overwrite").parquet("dbfs:/FileStore/tables/title_principals.parquet")
df_title_episode.write.mode("overwrite").parquet("dbfs:/FileStore/tables/title_episode.parquet")
df_title_ratings.write.mode("overwrite").parquet("dbfs:/FileStore/tables/title_ratings.parquet")

>> **Reload the saved dataframes** (do not need to run previous steps every time)

In [0]:
df_name_basics = spark.read.parquet("dbfs:/FileStore/tables/name_basics.parquet")
df_title_basics = spark.read.parquet("dbfs:/FileStore/tables/title_basics.parquet")
df_title_akas = spark.read.parquet("dbfs:/FileStore/tables/title_akas.parquet")
df_title_crew = spark.read.parquet("dbfs:/FileStore/tables/title_crew.parquet")
df_title_principals = spark.read.parquet("dbfs:/FileStore/tables/title_principals.parquet")
df_title_episode = spark.read.parquet("dbfs:/FileStore/tables/title_episode.parquet")
df_title_ratings = spark.read.parquet("dbfs:/FileStore/tables/title_ratings.parquet")

### Network Inference, Let’s build a network
In the following questions you will look to summarise the data and build a network. We want to examine a network that abstracts how actors and actress are related through their co-participation in movies. To that end perform the following steps:

**Q1** Create a DataFrame that combines **all the information** on each of the titles (i.e., movies, tv-shows, etc …) and **all of the information** the participants in those movies (i.e., actors, directors, etc … ), make sure the actual names of the movies and participants are included. It may be worth reviewing the following questions to see how this dataframe will be used.

How many rows does your dataframe have?

In [0]:
# We combine the dataframes df_title_principals + df_title_basics + df_name_basics + df_title_ratings(for later use)   [no akas, no episode]
# + crew? maybe extra bcs we have names?
# from Q2: title_principals, title_basics, name_basics
# from Q9: title_ratings

# Left join on title basics
df_Q1 = df_title_basics.join(df_title_ratings, "tconst", "left").join(df_title_crew, "tconst", "left").join(df_title_principals, "tconst", "left").join(df_name_basics, "nconst", "left")

## save the dataframe for later use
#df_Q1.write.mode("overwrite").parquet("dbfs:/FileStore/tables/df_Q1.parquet")
## to reload
#df_Q1 = spark.read.parquet("dbfs:/FileStore/tables/df_Q1.parquet")

In [0]:
# number of rows
df_Q1.count() # same output with or without including df_title_crew

Out[7]: 87259312

#### Q1 ANSWER: the created dataframe has *number* rows.

**Q2** Create a new DataFrame based on the previous step, with the following removed:
1. Any participant that is not an actor or actress (as measured by the category column);
1. All adult movies;
1. All dead actors or actresses;
1. All actors or actresses born before 1920 or with no date of birth listed;
1. All titles that are not of the type movie.

How many rows does your dataframe have?

In [0]:
# a bit of data exploration on categories from df_title_principals

df_title_principals.groupBy("category").count().orderBy('count').show() # see all categories of jobs

+-------------------+--------+
|           category|   count|
+-------------------+--------+
|      archive_sound|    7208|
|    archive_footage|  505595|
|   casting_director| 1008102|
|production_designer| 1054611|
|           composer| 2836970|
|    cinematographer| 3487347|
|             editor| 4559715|
|           producer| 6543892|
|           director| 7472221|
|             writer|10462168|
|               self|12133620|
|            actress|15484039|
|              actor|20676963|
+-------------------+--------+



In [0]:
# a bit of data exploration on deathYear from df_name_basics

from pyspark.sql.functions import isnan, when, count, col, regexp_extract

# count the number of null or NaN values
null_count = df_name_basics.select(count(when(isnan("deathYear") | col("deathYear").isNull(), "deathYear")).alias("null_count")).collect()[0]["null_count"]
print(f"Number of null values in 'deathYear': {null_count}")

# display the set of non-numeric values for deathYear
non_digit_values = df_name_basics.filter(~col("deathYear").rlike("^[0-9]+$"))
non_digit_values.select("deathYear").distinct().show()

Number of null values in 'deathYear': 0
+---------+
|deathYear|
+---------+
|       \N|
+---------+



In [0]:
df_Q2= df_Q1.filter(
    ( (df_Q1["category"]=="actor") | (df_Q1["category"]=="actress") ) &
    (df_Q1["isAdult"] == 0) &
    (df_Q1["deathYear"] == r"\N") &   # using r for raw string
    (df_Q1["birthYear"] >= 1920) &
    (df_Q1["titleType"] == "movie")
)
## save the dataframe for later use
#df_Q2.write.mode("overwrite").parquet("dbfs:/FileStore/tables/df_Q2.parquet")
## to reload
#df_Q2 = spark.read.parquet("dbfs:/FileStore/tables/df_Q2.parquet")

In [0]:
df_Q2.count()

Out[5]: 930646

#### Q2 ANSWER: the created dataframe has *number* rows.

**Q3** Convert the above Dataframe to an RDD. Use map and reduce to create a paired RDD which counts how many movies each actor / actress appears in.

Display names of the top 10 actors/actresses according to the number of movies in which they appeared. Be careful to deal with different actors / actresses with the same name, these could be different people.

In [0]:
# convert DataFrame to RDD
df_Q2_rdd = df_Q2.rdd

In [0]:
#*nconst instead of primaryName in order to avoid losing info due to actors/actresses with the same name

# pair RDD
actor_appearances_rdd = df_Q2_rdd.map(lambda row: (row["nconst"], 1))
# count appearances of each actor/actress
actor_counts_rdd = actor_appearances_rdd.reduceByKey(lambda a, b: a + b)

sorted_actor_counts_rdd = actor_counts_rdd.sortBy(lambda x: x[1], ascending=False)
top_10_actors = sorted_actor_counts_rdd.take(10)

In [0]:
# Display the names of the top 10 actors/actresses
top_10_actors_names = [df_name_basics.filter(df_name_basics["nconst"] == actor[0]).select("primaryName").first()[0] for actor in top_10_actors]
for i, actor_name in enumerate(top_10_actors_names, start=1):
    print(f"{i}. {actor_name}: {top_10_actors[i-1][1]} movies")

1. Brahmanandam: 1130 movies
2. Jagathy Sreekumar: 659 movies
3. Shakti Kapoor: 600 movies
4. Eric Roberts: 492 movies
5. Aruna Irani: 467 movies
6. Nassar: 440 movies
7. Mammootty: 437 movies
8. Helen: 433 movies
9. Tanikella Bharani: 412 movies
10. Mohanlal: 409 movies


#### Q3 ANSWER: top 10 actors/actresses (according to the number of movies in which they appeared):
1. Brahmanandam: 1130 movies
2. Jagathy Sreekumar: 659 movies
3. Shakti Kapoor: 600 movies
4. Eric Roberts: 492 movies
5. Aruna Irani: 467 movies
6. Nassar: 440 movies
7. Mammootty: 437 movies
8. Helen: 433 movies
9. Tanikella Bharani: 412 movies
10. Mohanlal: 409 movies

**Q4** Start with the dataframe from Q2. Generate a DataFrame that lists all links of your network. Here we shall consider that a link connects a pair of actors/actresses if they participated in at least one movie together (actors / actresses should be represented by their unique ID's). For every link we then need anytime a pair of actors were together in a movie as a link in each direction (A -> B and B -> A). However links should be distinct we do not need duplicates when two actors worked together in several movies. 

Display a DataFrame with the first 10 edges.

In [0]:
from pyspark.sql.functions import col

# select columns of title-id and name-id 
df_links = df_Q2.select("tconst", "nconst")

# perform self-join to find links (=pairs of actors/actresses who participated in the same movie)
df_pairs = df_links.alias("a").join(df_links.alias("b"), "tconst")

# distinct links and remove duplicates
df_Q4_edges = df_pairs.filter(col("a.nconst") != col("b.nconst")).select("a.nconst", "b.nconst").distinct()

## save the dataframe for later use
#df_Q4_edges.write.mode("overwrite").parquet("dbfs:/FileStore/tables/df_Q4.parquet")
## to reload
#df_Q4_edges = spark.read.parquet("dbfs:/FileStore/tables/df_Q4.parquet")

df_Q4_edges.show(10)

+---------+---------+
|   nconst|   nconst|
+---------+---------+
|nm0566478|nm0388155|
|nm0001989|nm0001012|
|nm0723771|nm0555533|
|nm0280772|nm0744675|
|nm0000728|nm0265276|
|nm0846357|nm0171993|
|nm0513419|nm0000476|
|nm0789558|nm0447775|
|nm0248815|nm0302837|
|nm0089392|nm0610092|
+---------+---------+
only showing top 10 rows



Extra- displaying the real names:

### I think it's giving wrong names, we could remove since it's not asked

In [0]:
df_edges = df_pairs.filter(col("a.nconst") < col("b.nconst")).select(col("a.nconst").alias("nconst1"), col("b.nconst").alias("nconst2")).distinct()

# real name of the first actor/actress
df_edges_with_names1 = df_edges.join(df_name_basics.select(col("nconst").alias("nconst1"), col("primaryName").alias("name1")), "nconst1")
# real name of the second actor/actress
df_edges_with_names2 = df_edges_with_names1.join(df_name_basics.select(col("nconst").alias("nconst2"), col("primaryName").alias("name2")), "nconst2")

df_edges_with_real_names = df_edges_with_names2.select("name1", "name2")

# Display the DataFrame with the first 10 edges
df_edges_with_real_names.show(10, truncate=False)

+-----------------+--------------+
|name1            |name2         |
+-----------------+--------------+
|Gillian Anderson |Rowan Atkinson|
|Brad Pitt        |Sandra Bullock|
|Dan Aykroyd      |John Cusack   |
|Matthew Broderick|John Cusack   |
|Nicolas Cage     |John Cusack   |
|Steve Buscemi    |John Cusack   |
|Gong Li          |John Cusack   |
|John Cleese      |John Cusack   |
|Dan Aykroyd      |David Duchovny|
|Brad Pitt        |David Duchovny|
+-----------------+--------------+
only showing top 10 rows



#### Q4 ANSWER: 10 first pairs of actors/actresses who participated in the same movie in the dataframe 'df_edges'


| nconst     | nconst     |
|------------|------------|
|nm0566478|nm0388155|
|nm0001989|nm0001012|
|nm0723771|nm0555533|
|nm0280772|nm0744675|
|nm0000728|nm0265276|
|nm0846357|nm0171993|
|nm0513419|nm0000476|
|nm0789558|nm0447775|
|nm0248815|nm0302837|
|nm0089392|nm0610092|



**Q5** Compute the page rank of each actor. This can be done using GraphFrames or
by using RDDs and the iterative implementation of the PageRank algorithm. Do not take
more than 5 iterations and use reset probility = 0.1.

List the top 10 actors / actresses by pagerank.

*Install graphframes library: clusters -> click on working cluster -> "Libraries" tab -> "Install New" -> "Maven" -> in coordinates enter 'graphframes:graphframes:0.8.1-spark3.0-s_2.12 -> install

In [0]:
# using GraphFrames #
from graphframes import GraphFrame
from pyspark.sql.functions import col

# parameters:
reset_probability = 0.1
iterations = 5
# df_edges from Q4

# Step 1: create graph of vertices and edges
#         create vertices df with distinct actors/actresses & create edges df with links between actors/actresses
vertices = df_Q4_edges.select(col("a.nconst").alias("id")).distinct()
edges = df_Q4_edges.select(col("a.nconst").alias("src"), col("b.nconst").alias("dst")).distinct()

graph_Q5 = GraphFrame(vertices, edges)

# Step 2: Compute PageRank
pageranks_Q5 = graph_Q5.pageRank(resetProbability=reset_probability, maxIter=iterations)

/databricks/spark/python/pyspark/sql/dataframe.py:170: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(
/databricks/spark/python/pyspark/sql/dataframe.py:149: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


In [0]:
# Step 3: Join with df_name_basics to get real names
pageranks_with_names = pageranks_Q5.vertices.join(df_name_basics, pageranks_Q5.vertices.id == df_name_basics.nconst) \
                                            .select("primaryName", "pagerank") \
                                            .orderBy(col("pagerank").desc())

# Display the top 10 actors/actresses by PageRank
pageranks_with_names.show(10, truncate=False)

+----------------+------------------+
|primaryName     |pagerank          |
+----------------+------------------+
|Eric Roberts    |58.67753663800431 |
|Michael Madsen  |31.64025706185349 |
|Danny Trejo     |24.784419492571914|
|Keith David     |23.14300297927517 |
|Michael Paré    |22.730506131072694|
|Joe Estevez     |22.295995558628736|
|Richard Riehle  |21.416559204115934|
|Malcolm McDowell|21.288954359634516|
|Lloyd Kaufman   |21.2516153454063  |
|Lance Henriksen |20.77223620331601 |
+----------------+------------------+
only showing top 10 rows



#### Q5 ANSWER: The 10 most influential actors/actresses within the IMDb network (pagerank) :


|primaryName |pagerank    |
|------------|------------|
|Eric Roberts    |58.67753663800431 |
|Michael Madsen  |31.64025706185349 |
|Danny Trejo     |24.784419492571914|
|Keith David     |23.14300297927517 |
|Michael Paré    |22.730506131072694|
|Joe Estevez     |22.295995558628736|
|Richard Riehle  |21.416559204115934|
|Malcolm McDowell|21.288954359634516|
|Lloyd Kaufman   |21.2516153454063  |
|Lance Henriksen |20.77223620331601 |

**Q6**: Create an RDD with the number of outDegrees for each actor. Display the top 10 by outdegrees.

In [0]:
Q6_outdegrees = graph_Q5.outDegrees
Q6_names = Q6_outdegrees.join(df_name_basics, Q6_rdd.id == df_name_basics.nconst)
# display
Q6_names.select("id", "primaryName", "outDegree").orderBy(col("outDegree").desc()).show(10)

/databricks/spark/python/pyspark/sql/dataframe.py:149: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


+---------+----------------+---------+
|       id|     primaryName|outDegree|
+---------+----------------+---------+
|nm0000616|    Eric Roberts|     1337|
|nm0000514|  Michael Madsen|      841|
|nm0451600|     Anupam Kher|      760|
|nm0202966|     Keith David|      706|
|nm0410902| Renji Ishibashi|      703|
|nm0621937|          Nassar|      688|
|nm0000367|Gérard Depardieu|      676|
|nm0001803|     Danny Trejo|      663|
|nm0256628|     Akira Emoto|      658|
|nm0695177|     Prakash Raj|      648|
+---------+----------------+---------+
only showing top 10 rows



#### Q6 ANSWER: The 10 actors/actresses based on the number of collaborations (movies) they've been involved (outdegrees):

|       id|     primaryName|outDegree|
|---------|----------------|---------|
|nm0000616|    Eric Roberts|     1337|
|nm0000514|  Michael Madsen|      841|
|nm0451600|     Anupam Kher|      760|
|nm0202966|     Keith David|      706|
|nm0410902| Renji Ishibashi|      703|
|nm0621937|          Nassar|      688|
|nm0000367|Gérard Depardieu|      676|
|nm0001803|     Danny Trejo|      663|
|nm0256628|     Akira Emoto|      658|
|nm0695177|     Prakash Raj|      648|

### Let’s play Kevin’s own game

**Q7** Start with the graphframe / dataframe you developed in the previous questions. Using Spark GraphFrame and/or Spark Core library perform the following steps:

1. Identify the id of Kevin Bacon, there are two actors named ‘Kevin Bacon’, we will use the one with the highest degree, that is, the one that participated in most titles;
1. Estimate the shortest path between every actor in the database actors and Kevin Bacon, keep a dataframe with this information as you will need it later;
1. Summarise the data, that is, count the number of actors at each number of degress from kevin bacon (you will need to deal with actors unconnected to kevin bacon, if not connected to Kevin Bacon given these actors / actresses a score/degree of 20).

In [0]:
#### 1 ####
from pyspark.sql.functions import col

# get primaryName
Q6_names = Q6_outdegrees.join(df_name_basics, Q6_outdegrees.id == df_name_basics.nconst)

# filter for actors named "Kevin Bacon" and order by outDegree
df_kevin_bacon = Q6_names.filter(Q6_names.primaryName == "Kevin Bacon").orderBy(col("outDegree").desc())

# Display id and name of Kevin Bacon actor with the highest outDegree
df_kevin_bacon.select("id", "primaryName").show(1)

+---------+-----------+
|       id|primaryName|
+---------+-----------+
|nm0000102|Kevin Bacon|
+---------+-----------+



In [0]:
#### 2 ####
kevin_bacon_id = kevin_bacon_df.select("id").first()["id"]

df_shortest_paths = graph_Q5.shortestPaths(landmarks=[kevin_bacon_id])

/databricks/spark/python/pyspark/sql/dataframe.py:149: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


In [0]:
#### 3 ####
from pyspark.sql.functions import map_values, concat_ws

# dataframe with id, distance
df_Q7_3 = df_shortest_paths.select('id','distances')
# get the distance from a map type column
df_Q7_3 = df_Q7_3.withColumn('distances', concat_ws(',',map_values("distances")))

df_Q7_3_impute = df_Q7_3.replace('', '20') # imputes value 20 for actors with no connection to Kevin Bacon

# number of actors with each distance
df_Q7_3_impute.groupBy('distances').count().orderBy('count', ascending=False).show()

+---------+-----+
|distances|count|
+---------+-----+
|        3|58553|
|        4|42446|
|        2|14169|
|        5| 4850|
|       20| 4605|
|        6|  510|
|        1|  354|
|        7|   56|
|        8|   20|
|        9|    3|
|        0|    1|
+---------+-----+



#### Q7 ANSWER: 
##### 1) 
|       id|primaryName|
|---------|-----------|
|nm0000102|Kevin Bacon|

##### 2) Created
df_shortest_paths

##### 3) 
|distances|count|
|---------|-----|
|        3|58553|
|        4|42446|
|        2|14169|
|        5| 4850|
|       20| 4605|
|        6|  510|
|        1|  354|
|        7|   56|
|        8|   20|
|        9|    3|
|        0|    1|

### Exploring the data with RDD's

Using RDDs and (not dataframes) answer the following questions (if you loaded your data into spark in a dataframe you can convert to an RDD of rows easily using `.rdd`):

**Q8** Movies can have multiple genres. Considering only titles of the type 'movie' what is the combination of genres that is the most popluar (as measured by number of reviews). Hint: paired RDD's will be useful.

**Q9** Movies can have multiple genres. Considering only titles of the type 'movie', and movies with more than 400 ratings, what is the combination of genres that has the highest **average movie rating** (you can average the movie rating for each movie in that genre combination). Hint: paired RDD's will be useful.

**Q10** Movies can have multiple genres. What is **the individual genre** which is the most popular as meaured by number of votes. Votes for multiple genres count towards each genre listed. Hint: flatmap and pairedRDD's will be useful here.

## Engineering the perfect cast
We have created a number of potential features for predicting the rating of a movie based on its cast. Use sparkML to build a simple linear model to predict the rating of a movie based on the following features:

1. The total number of movies in which the actors / actresses have acted (based on Q3)
1. The average pagerank of the cast in each movie (based on Q5)
1. The average outDegree of the cast in each movie (based on Q6)
1. The average value for for the cast of degrees of Kevin Bacon (based on Q7).

You will need to create a dataframe with the required features and label. Use a pipeline to create the vectors required by sparkML and apply the model. Remember to split your dataset, leave 30% of the data for testing, when splitting your data use the option seed=0.

**Q11** Provide the coefficients of the regression and the accuracy of your model on that test dataset according to RSME.

**Q12** What score would your model predict for the 1997 movie Titanic.

**Q13** Create dummy variables for each of the top 10 movie genres for Q10. These variable should have a value of 1 if the movie was rated with that genre and 0 otherwise. For example the 1997 movie Titanic should have a 1 in the dummy variable column for Romance, and a 1 in the dummy variable column for Drama, and 0's in all the other dummy variable columns.

Does adding these variable to the regression improve your results? What is the new RMSE and predicted rating for the 1997 movie Titanic.

**Q14 - Open Question**: Improve your model by testing different machine learning algorithms, using hyperparameter tuning on these algorithms, changing the included features. What is the RMSE of you final model and what rating does it predict for the 1997 movie Titanic.